In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf='gg2013.json')
# df = pd.read_json(path_or_buf='gg2015.json')

In [3]:
data = df['text']

In [4]:
# sample data if necessary
sample_size = 200000
if len(df) > sample_size:
    data = data.sample(n=sample_size)
len(data)

174643

In [5]:
import json
with open('autograder/gg2013answers.json') as file:
    ground_truth = json.load(file)
file.closed

True

In [6]:
# test
true_award_winners = ground_truth['award_data']
awards_old = sorted([item[0] for item in true_award_winners.items()])

for name in awards_old:
    print(name)
    print('true winner:', true_award_winners[name]['winner'])
    print()

best animated feature film
true winner: brave

best director - motion picture
true winner: ben affleck

best foreign language film
true winner: amour

best mini-series or motion picture made for television
true winner: game change

best motion picture - comedy or musical
true winner: les miserables

best motion picture - drama
true winner: argo

best original score - motion picture
true winner: life of pi

best original song - motion picture
true winner: skyfall

best performance by an actor in a mini-series or motion picture made for television
true winner: kevin costner

best performance by an actor in a motion picture - comedy or musical
true winner: hugh jackman

best performance by an actor in a motion picture - drama
true winner: daniel day-lewis

best performance by an actor in a supporting role in a motion picture
true winner: christoph waltz

best performance by an actor in a supporting role in a series, mini-series or motion picture made for television
true winner: ed harris


In [7]:
# hard-coded awards 2013-2015
awards_old

['best animated feature film',
 'best director - motion picture',
 'best foreign language film',
 'best mini-series or motion picture made for television',
 'best motion picture - comedy or musical',
 'best motion picture - drama',
 'best original score - motion picture',
 'best original song - motion picture',
 'best performance by an actor in a mini-series or motion picture made for television',
 'best performance by an actor in a motion picture - comedy or musical',
 'best performance by an actor in a motion picture - drama',
 'best performance by an actor in a supporting role in a motion picture',
 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television',
 'best performance by an actor in a television series - comedy or musical',
 'best performance by an actor in a television series - drama',
 'best performance by an actress in a mini-series or motion picture made for television',
 'best performance by an actress in a motion

In [8]:
# references: https://en.wikipedia.org/wiki/70th_Golden_Globe_Awards
# hard-coded awards winners
winners = ['Argo', 'Les Miserables', 'Jessica Chastain', 'Daniel Day-Lewis', 'Jennifer Lawrence', 'Hugh Jackman', 
           'Anne Hathaway', 'Christoph Waltz',
           'Ben Affleck', 'Quentin Tarantino', 'Brave', 'Amour', 
           'Skyfall', 'Life of Pi', 
           'Homeland', 'Girls', 'Game Change', 
           'Julianne Moore', 'Kevin Costner', 
           'Claire Danes', 'Damian Lewis', 'Lena Dunham', 'Don Cheadle',
           'Maggie Smith', 'Ed Harris', 
           'Jodie Foster']
print(len(winners))

26


In [9]:
# hard-coded awards 2018-2019
awards = ['Best Motion Picture - Drama',
         'Best Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Motion Picture - Drama',
         'Best Performance by an Actor in a Motion Picture - Drama',
         'Best Performance by an Actress in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actor in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in any Motion Picture',
         'Best Performance by an Actor in a Supporting Role in any Motion Picture',
         'Best Director - Motion Picture',
         'Best Screenplay - Motion Picture',
         'Best Motion Picture - Animated',
         'Best Motion Picture - Foreign Language',
         'Best Original Score - Motion Picture',
         'Best Original Song - Motion Picture',
         'Best Television Series - Drama',
         'Best Television Series - Musical or Comedy',
         'Best Television Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actress in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actor in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actress in A Television Series - Drama',
         'Best Performance by an Actor in A Television Series - Drama',
         'Best Performance by an Actress in a Television Series - Musical or Comedy',
         'Best Performance by an Actor in a Television Series - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actor in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Cecil B. deMille Award']
print(len(awards))

26


In [10]:
import re

In [11]:
def reduce(line):
    # test
    # replace "motion picture" to "movie"
#     pattern = r'\bmotion picture\b'
#     line = re.sub(pattern, 'movie', line.lower())
    
    # replace "television" to "tv"
    pattern = r'\btelevision\b'
    line = re.sub(pattern, 'tv', line.lower())

    # remove words "best", "performance", "motion", "picture", "limited", "language", "role", in", "a", "an", "any", "made", "for", "by", "b.", "award", and all punctuations
    pattern = r'\bbest\b|\bperformance\b|\bmotion\b|\bpicture\b|\blimited\b|\blanguage\b|\brole\b|\bin\b|\ba\b|\ban\b|\bany\b|\bmade\b|\bfor\b|\bby\b|\bb\b|\baward\b|[^\w\s]'
    return re.sub(pattern, ' ', line.lower()).split()

In [12]:
# test reduce function on old awards
for award in awards_old:
    ls = reduce(award)
    print(sorted(set(ls), key=lambda word: ls.index(word)))

['animated', 'feature', 'film']
['director']
['foreign', 'film']
['mini', 'series', 'or', 'tv']
['comedy', 'or', 'musical']
['drama']
['original', 'score']
['original', 'song']
['actor', 'mini', 'series', 'or', 'tv']
['actor', 'comedy', 'or', 'musical']
['actor', 'drama']
['actor', 'supporting']
['actor', 'supporting', 'series', 'mini', 'or', 'tv']
['actor', 'tv', 'series', 'comedy', 'or', 'musical']
['actor', 'tv', 'series', 'drama']
['actress', 'mini', 'series', 'or', 'tv']
['actress', 'comedy', 'or', 'musical']
['actress', 'drama']
['actress', 'supporting']
['actress', 'supporting', 'series', 'mini', 'or', 'tv']
['actress', 'tv', 'series', 'comedy', 'or', 'musical']
['actress', 'tv', 'series', 'drama']
['screenplay']
['tv', 'series', 'comedy', 'or', 'musical']
['tv', 'series', 'drama']
['cecil', 'demille']


In [13]:
def generate_award_num_keywords_map(awards):
    awards_reduced = []
    award_num_keywords_map = {}
    for i, award in enumerate([sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]):
        if 'or' in award:
            # find number of words from "or" to the end of award name
            num_words = len(award) - 1 - award.index('or')
            award.remove('or')
            # do not count words from "or" to the end of award name
            award_num_keywords_map[i] = len(award) - num_words             
        else:
            award_num_keywords_map[i] = len(award)
        
#         if 'movie' in award:
#             award_num_keywords_map[i] -= 1
            
        awards_reduced.append(award)
    return award_num_keywords_map, awards_reduced


award_num_keywords_map, awards_reduced = generate_award_num_keywords_map(awards)

In [14]:
awards_reduced

[['drama'],
 ['musical', 'comedy'],
 ['actress', 'drama'],
 ['actor', 'drama'],
 ['actress', 'musical', 'comedy'],
 ['actor', 'musical', 'comedy'],
 ['actress', 'supporting'],
 ['actor', 'supporting'],
 ['director'],
 ['screenplay'],
 ['animated'],
 ['foreign'],
 ['original', 'score'],
 ['original', 'song'],
 ['tv', 'series', 'drama'],
 ['tv', 'series', 'musical', 'comedy'],
 ['tv', 'series'],
 ['actress', 'series', 'tv'],
 ['actor', 'series', 'tv'],
 ['actress', 'tv', 'series', 'drama'],
 ['actor', 'tv', 'series', 'drama'],
 ['actress', 'tv', 'series', 'musical', 'comedy'],
 ['actor', 'tv', 'series', 'musical', 'comedy'],
 ['actress', 'supporting', 'series', 'tv'],
 ['actor', 'supporting', 'series', 'tv'],
 ['cecil', 'demille']]

In [15]:
# test
'Motion Picture' in "aaasdfasdfMotion Picturesasdfasdfasdf"

True

In [16]:
def describe_awards(awards, awards_reduced, award_num_keywords_map):
    pairs = sorted(award_num_keywords_map.items(), key=lambda pair: pair[0])
    for pair in pairs:
        print(pair[0])
        print(awards[pair[0]])
        print(awards_reduced[pair[0]])
        if pair[1] < len(awards_reduced[pair[0]]):
            print('******')
        print(pair) # (index, num_keywords_to_match)    
        print()
        
describe_awards(awards, awards_reduced, award_num_keywords_map)

0
Best Motion Picture - Drama
['drama']
(0, 1)

1
Best Motion Picture - Musical or Comedy
['musical', 'comedy']
******
(1, 1)

2
Best Performance by an Actress in a Motion Picture - Drama
['actress', 'drama']
(2, 2)

3
Best Performance by an Actor in a Motion Picture - Drama
['actor', 'drama']
(3, 2)

4
Best Performance by an Actress in a Motion Picture - Musical or Comedy
['actress', 'musical', 'comedy']
******
(4, 2)

5
Best Performance by an Actor in a Motion Picture - Musical or Comedy
['actor', 'musical', 'comedy']
******
(5, 2)

6
Best Performance by an Actress in a Supporting Role in any Motion Picture
['actress', 'supporting']
(6, 2)

7
Best Performance by an Actor in a Supporting Role in any Motion Picture
['actor', 'supporting']
(7, 2)

8
Best Director - Motion Picture
['director']
(8, 1)

9
Best Screenplay - Motion Picture
['screenplay']
(9, 1)

10
Best Motion Picture - Animated
['animated']
(10, 1)

11
Best Motion Picture - Foreign Language
['foreign']
(11, 1)

12
Best Orig

In [17]:
# tweaking magic
awards_reduced[0].append('movie')
awards_reduced[1].append('movie')
awards_reduced[2].append('movie')
awards_reduced[11].append('language')

describe_awards(awards, awards_reduced, award_num_keywords_map)

0
Best Motion Picture - Drama
['drama', 'movie']
******
(0, 1)

1
Best Motion Picture - Musical or Comedy
['musical', 'comedy', 'movie']
******
(1, 1)

2
Best Performance by an Actress in a Motion Picture - Drama
['actress', 'drama', 'movie']
******
(2, 2)

3
Best Performance by an Actor in a Motion Picture - Drama
['actor', 'drama']
(3, 2)

4
Best Performance by an Actress in a Motion Picture - Musical or Comedy
['actress', 'musical', 'comedy']
******
(4, 2)

5
Best Performance by an Actor in a Motion Picture - Musical or Comedy
['actor', 'musical', 'comedy']
******
(5, 2)

6
Best Performance by an Actress in a Supporting Role in any Motion Picture
['actress', 'supporting']
(6, 2)

7
Best Performance by an Actor in a Supporting Role in any Motion Picture
['actor', 'supporting']
(7, 2)

8
Best Director - Motion Picture
['director']
(8, 1)

9
Best Screenplay - Motion Picture
['screenplay']
(9, 1)

10
Best Motion Picture - Animated
['animated']
(10, 1)

11
Best Motion Picture - Foreign L

In [18]:
retweets_freq_dict = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @([\w\'/]*)\b: ') 
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT @' prefix
        string = match.group()[4:]
        if string in retweets_freq_dict:
            retweets_freq_dict[string] += 1
        else:
            retweets_freq_dict[string] = 1
    return re.sub(pattern, ' ', line)

In [19]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [20]:
at_freq_dict = {}

def remove_at(line):
    pattern = re.compile(r'@([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in at_freq_dict:
                at_freq_dict[match] += 1
            else:
                at_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [21]:
def remove_url(line):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b')
    return re.sub(pattern, ' ', line)

In [22]:
def cleanse(line):
    # replace everything to ' ' except alphanumeric character, whitespace, apostrophe, hashtag
    return re.sub(r'[^\w\s\'#]', ' ', line)

In [23]:
%%time
cleansed_data = []
for tweet in data:
    line = remove_retweet_prefix(tweet)
    line = remove_hashtag(line)
    line = remove_at(line)
    line = remove_url(line)
    line = cleanse(line)
    cleansed_data.append(line)

CPU times: user 3.51 s, sys: 33.2 ms, total: 3.55 s
Wall time: 3.54 s


In [24]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [25]:
def identify_entities(text):
    entities = list(nlp(text).ents)
    tags = {}
    for entity in entities:
        if entity not in tags:
            tags[entity.text]=[entity.label_]
    return tags

In [26]:
# some ideas to improve spaCy's Name Entity Recognizer
# https://github.com/explosion/spaCy/issues/1655
# https://github.com/explosion/spacy/blob/master/examples/training/train_new_entity_type.py
# https://spacy.io/usage/training#section-ner

In [27]:
def check_target_words(awards_reduced):
    target_words = ['actor', 'actress', 'director', 'score', 'song', 'foreign', 'tv']
    for word in target_words:
        if word in awards_reduced:
            return word
    return False

def check_primary_target_words(awards_reduced):
    target_words = ['actor', 'actress', 'director', 'score', 'song']
    for word in target_words:
        if word in awards_reduced:
            return word
    return False

check_target_words(awards_reduced[12])

'score'

In [28]:
from fuzzywuzzy import fuzz

def remove_goldeb_globes(top_results, entity_freq_dict):
    golden_globes = [name for name in [pair[0] for pair in top_results] if fuzz.ratio(name.lower(), 'golden globes') > 60]
    for name in golden_globes:
        if name in entity_freq_dict:
            del entity_freq_dict[name]
    return entity_freq_dict


# we also consider dropping all lower cases examples or examples that contain digit(s), which are not names
def filter_names(pair_list, entity_freq_dict):
    filtered_results = []
    for pair in pair_list:
        string = ''.join(pair[0].split())
        if not all(char.islower() for char in string) and not any(char.isdigit() for char in string):
            filtered_results.append(pair)
        else:
            if pair[0] in entity_freq_dict:
                del entity_freq_dict[pair[0]]
    return filtered_results, entity_freq_dict


def merge_names(top_results, entity_freq_dict):
    names = [pair[0] for pair in top_results]
    names_clusters = []
    for name in names:
        # each name starts as a cluster
        cluster = [name]
        names_to_reduce = names[:]
        names_to_reduce.remove(name)
        # one vs. all comparisons
        for i in names_to_reduce:
            ratio = fuzz.ratio(name.lower(), i.lower())
            # if similarity is larger than 75 or one name is contained in the other name
            if ratio > 75 or re.search(name, i, flags=re.IGNORECASE) or re.search(i, name, flags=re.IGNORECASE):
                cluster.append(i)
        # if multiple names are identified in one cluster
        if len(cluster) > 1:
            names_clusters.append(cluster)

    # find names clusters that should merge
    # ['Amy Poehler', 'Amy', 'Amy Poelher']
    # ['Tina', 'Tina Fey']

    # sort clusters
    names_clusters.sort()
    # sort within each cluster
    names_clusters = ['|'.join(sorted(cluster)) for cluster in names_clusters]
    # remove overlaps
    names_clusters_reduced = [line.split('|') for line in list(set(names_clusters))]
    # sort by length from shortest to longest (merge from the shortest)
    names_clusters_reduced.sort(key=len)

    # weighted frequency of an entity is defined by its frequency multiplied by its string length
    def weighted_freq(element):
        return entity_freq_dict[element] * len(element)

    e = entity_freq_dict.copy()
    for cluster in names_clusters_reduced:
        # select the entity name with highest weighted frequency
        selected_entity_name = max(cluster, key=weighted_freq)
        cluster.remove(selected_entity_name)
        # for names to be merged to the selected entity name
        for name in cluster:
            # if not deleted in previous cases, cumulate frequencies to the selected entity
            if name in e and selected_entity_name in e:
                e[selected_entity_name] += e[name]
                # reward merging
                e[selected_entity_name] += round(e[selected_entity_name]*1/(names.index(selected_entity_name)+1-0.8) / e[name])
                del e[name]

    top_10 = sorted(e.items(), key=lambda pair: pair[1], reverse=True)[:10]
    return top_10

In [29]:
def find_award_winner(awards, awards_reduced, award_index, verbose=False):
    print(award_index)
    print('Predicting for:', awards[award_index])
    print('true winner:', winners[award_index])
    award = awards_reduced[award_index]
    print('award name reduced:', award)
    num_keywords_to_match = award_num_keywords_map[award_index]
    print("num keywords to match:", num_keywords_to_match)
    
    # add word boundary '\b' to prevent grabbing examples like "showing" and "wonder"    
    pattern = re.compile(r'(\bwin)|(\bwon\b)|(\bbest\b)', re.IGNORECASE)
        
    entity_freq_dict = {}
    
    num = 0
    flag = 0
    target_word_pattern = None
    match_target_word = None
    
    target_word = check_target_words(award)
    
    # if len(award) != num_keywords_to_match (awards that have 'or' options)
    # and len(award) != 2 (for the case of ['musical', 'comedy']), 
    # in addition, if target word in award, target word must be matched
    if len(award) != num_keywords_to_match and target_word:
        flag = 1
        # find target word pattern to match for sure
        primary_target_word = check_primary_target_words(award)  
        if primary_target_word:
            target_word_pattern = re.compile(r'\b{0}\b'.format(primary_target_word), re.IGNORECASE)
            print("matching target word '{0}'".format(target_word))
        # else if primary keyword is not found, match for secondary keyword
        elif 'tv' in award:
            target_word_pattern = re.compile(r'\b{0}\b'.format('tv'), re.IGNORECASE)
            print("matching target word 'tv'")
        else:
            target_word_pattern = re.compile(r'\b{0}\b'.format(award[0]), re.IGNORECASE)
            print("matching target word '{0}'".format(award[0]))
                
        for line in cleansed_data:
            match = re.findall(pattern, line.lower())
            match_target_word = re.findall(target_word_pattern, line.lower())
            
            # if line contains at least number keywords to match and pattern is found
            num_keywords_matched = len(set(award).intersection(set(line.split())))
            if (match and num_keywords_matched >= num_keywords_to_match) and match_target_word:
                # reward longer match
                weight = 10 if any('win' in tup for tup in match) else 1
                ratio = round(num_keywords_matched/(num_keywords_to_match+0.0001))
                weight *= ratio
                
                tags = identify_entities(line)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(line)
                        print(tags.keys())
                        print()

                for entity in tags.keys():
                    entity_raw = entity
                    entity = entity.strip() 
                    if len(entity) > 1:
                        # add more weights for appropriate entity classification
                        if target_word in ['actor', 'actress', 'director'] and tags[entity_raw] == 'PERSON':
                            weight += 5
                        if target_word not in ['actor', 'actress', 'director'] and tags[entity_raw] == 'PERSON':
                            weight -= 5
                        if entity not in entity_freq_dict:
                            entity_freq_dict[entity] = weight
                        else:
                            entity_freq_dict[entity] += weight
                num += 1
    
    else:
        for line in cleansed_data:
            match = re.findall(pattern, line.lower())
            
            # if line contains at least number keywords to match and pattern is found
            num_keywords_matched = len(set(award).intersection(set(line.split())))
            if match and num_keywords_matched == num_keywords_to_match:
                
                weight = 10 if any('win' in tup for tup in match) else 1
                ratio = round(num_keywords_matched/(num_keywords_to_match+0.0001))
                weight *= ratio
                    
                tags = identify_entities(line)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(any('win' in tup for tup in match))
                        print(line)
                        print(tags.keys())
                        print()

                for entity in tags.keys():
                    entity_raw = entity
                    entity = entity.strip() 
                    if len(entity) > 1:
                        # add more weights for appropriate entity classification
                        if target_word in ['actor', 'actress', 'director'] and tags[entity_raw] == 'PERSON':
                            weight += 5
                        if target_word not in ['actor', 'actress', 'director'] and tags[entity_raw] == 'PERSON':
                            weight -= 5
                        if entity not in entity_freq_dict:
                            entity_freq_dict[entity] = weight
                        else:
                            entity_freq_dict[entity] += weight
                num += 1
    
    
    
    
    
    # if no results found, recursively add more keywords and reduce num_keywords_to_match
    while(num == 0 or num < 10):
        print('no results found or too few matches!!! add more keywords and reduce num_keywords_to_match!!!')
                
        num_keywords_to_match -= 1
        print("num keywords to match:", num_keywords_to_match)
        if num_keywords_to_match == 0:
            print('break loop!!!')
            break
        
        print('award before', award)
        # add alternative word for 'tv'
        if 'tv' in award:
            award.append('television')
        # add alternative word for 'Motion Picture'
        if 'Motion Picture' in awards[award_index]:
            award.append('movie')
            award.append('motion')
            award.append('picture')
        print('expanded award with more keywards', award)
        
        if flag == 1:
            if target_word:
                print("matching target word '{0}'".format(target_word))
            for line in cleansed_data:
                match = re.findall(pattern, line.lower())
                
                match_target_word = re.findall(target_word_pattern, line.lower())
                
                # if line contains at least number keywords to match and pattern is found
                num_keywords_matched = len(set(award).intersection(set(line.split())))
                if num_keywords_matched >= num_keywords_to_match and match and match_target_word:
                    
                    weight = 10 if any('win' in tup for tup in match) else 1
                    ratio = round(num_keywords_matched/(num_keywords_to_match+0.0001))
                    weight *= ratio
                    
                    tags = identify_entities(line)

                    if verbose:
                        # print the first 5 occurrences
                        if num < 5:
                            print(match)
                            print(line)
                            print(tags.keys())
                            print()

                    for entity in tags.keys():
                        entity_raw = entity
                        entity = entity.strip() 
                        if len(entity) > 1:
                            # add more weights for appropriate entity classification
                            if target_word in ['actor', 'actress', 'director'] and tags[entity_raw] == 'PERSON':
                                weight += 5
                            if target_word not in ['actor', 'actress', 'director'] and tags[entity_raw] == 'PERSON':
                                weight -= 5
                            if entity not in entity_freq_dict:
                                entity_freq_dict[entity] = weight
                            else:
                                entity_freq_dict[entity] += weight
                    num += 1
        else:
            for line in cleansed_data:
                match = re.findall(pattern, line.lower())
                # if line contains at least number keywords to match and pattern is found
                num_keywords_matched = len(set(award).intersection(set(line.split())))
                if num_keywords_matched == num_keywords_to_match and match:
                    
                    weight = 10 if any('win' in tup for tup in match) else 1
                    ratio = round(num_keywords_matched/(num_keywords_to_match+0.0001))
                    weight *= ratio
                    
                    tags = identify_entities(line)

                    if verbose:
                        # print the first 5 occurrences
                        if num < 5:
                            print(match)
                            print(line)
                            print(tags.keys())
                            print()

                    for entity in tags.keys():
                        entity_raw = entity
                        entity = entity.strip() 
                        if len(entity) > 1:
                            # add more weights for appropriate entity classification
                            if target_word in ['actor', 'actress', 'director'] and tags[entity_raw] == 'PERSON':
                                weight += 5
                            if target_word not in ['actor', 'actress', 'director'] and tags[entity_raw] == 'PERSON':
                                weight -= 5
                            if entity not in entity_freq_dict:
                                entity_freq_dict[entity] = weight
                            else:
                                entity_freq_dict[entity] += weight
                    num += 1
    print('num of matches:', num)
    return entity_freq_dict

# Predictions for Award Winners

In [30]:
%%time

# test by chaning key
key = 15
awards_reduced_copy = awards_reduced[:]

entity_freq_dict = find_award_winner(awards, awards_reduced_copy, key, verbose=False)
top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:100]
# remove 'golden globes' from identified host names
entity_freq_dict = remove_goldeb_globes(top_results, entity_freq_dict)
top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:30]
# filter for names if necessary
if 'actor' in awards_reduced[key] or 'actress' in awards_reduced[key] or 'director' in awards_reduced[key]:
    top_results, entity_freq_dict = filter_names(top_results, entity_freq_dict)
print('top results:')
print(top_results)
print('top results after merging:')
top_10 = merge_names(top_results, entity_freq_dict)
print(top_10)

15
Predicting for: Best Television Series - Musical or Comedy
true winner: Girls
award name reduced: ['tv', 'series', 'musical', 'comedy']
num keywords to match: 3
matching target word 'tv'
num of matches: 95
top results:
[('Don Cheadle', 156), ('Lena Dunham', 104), ('Girls', 91), ('Best Actress', 20), ('House of Lies', 14), ('for Girls', 10), ('RT', 10), ('Middle school', 10), ('Mono', 10), ('Comedy', 10), ('HAZZAH', 10), ('1 fab night', 10), ('BestActress', 10), ('Rock', 10), ('Amy Poehler', 4), ('George Clooney', 4), ('House', 1), ('Jay Leno', 1), ('Jimmy Fallon', 1), ('Chad Lowe  Cheers', 1)]
top results after merging:
[('Don Cheadle', 156), ('Girls', 106), ('Lena Dunham', 104), ('Best Actress', 31), ('House of Lies', 19), ('RT', 10), ('Middle school', 10), ('Mono', 10), ('Comedy', 10), ('HAZZAH', 10)]
CPU times: user 9.99 s, sys: 1.19 s, total: 11.2 s
Wall time: 2.59 s


In [31]:
%%time

award_winner_dict = {}
for key in award_num_keywords_map.keys():
    entity_freq_dict = find_award_winner(awards, awards_reduced, key, verbose=False)
    top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:100]
    # remove 'golden globes' from identified host names
    entity_freq_dict = remove_goldeb_globes(top_results, entity_freq_dict)
    top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:30]
    # filter for names if necessary
    if 'actor' in awards_reduced[key] or 'actress' in awards_reduced[key] or 'director' in awards_reduced[key]:
        top_results, entity_freq_dict = filter_names(top_results, entity_freq_dict)
    print('top results:')
    print(top_results)
    print('top results after merging:')
    top_10 = merge_names(top_results, entity_freq_dict)
    print(top_10)
    
    if len(top_10) != 0:
        award_winner_dict[awards[key]] = top_10[0][0]
    else:
        award_winner_dict[awards[key]] = ''
    print()

print(award_winner_dict)

0
Predicting for: Best Motion Picture - Drama
true winner: Argo
award name reduced: ['drama', 'movie']
num keywords to match: 1
num of matches: 1393
top results:
[('Homeland', 1325), ('Daniel Day Lewis', 549), ('Claire Danes', 500), ('Lincoln', 496), ('first', 414), ('Jessica Chastain', 364), ('Argo', 349), ('Julianne Moore', 339), ("Les Miz'", 300), ('Lewis  Chastain', 290), ('Damien Lewis', 264), ('15', 220), ('Ben Affleck', 146), ('Jennifer Lawrence', 141), ('for', 121), ('Zero Dark Thirty', 120), ('Kevin Costner', 112), ('Damian Lewis', 106), ('Django', 103), ('Kathryn Bigelow', 100), ('Latest', 100), ('Skyfall', 98), ('Adele', 94), ('Breaking', 90), ('12 year old', 90), ('RT', 84), ('Zero', 84), ('A lanky', 80), ('Game Change', 76), ('HBO', 72)]
top results after merging:
[('Homeland', 1325), ('Daniel Day Lewis', 549), ('Claire Danes', 500), ('Lincoln', 496), ('first', 414), ('Damien Lewis', 370), ('Jessica Chastain', 364), ('Argo', 349), ('Julianne Moore', 339), ("Les Miz'", 300)

num of matches: 137
top results:
[('Quentin Tarantino', 536), ('Django Unchained', 139), ('Tarantino', 64), ('Django', 48), ('WINNER', 40), ('Except', 40), ('Quentin', 22), ('QT', 21), ('tonight', 21), ('Lincoln', 11), ('quentin', 10), ('Very', 10), ('2nd', 10), ('Surprised', 10), ('Wanna', 10), ('one day', 10), ('XD', 10), ('Basterds', 10), ('3028', 10), ("Django's", 10), ('Will Lincoln', 10), ('as much as thought', 10), ('Argo', 10), ("Kerri Washington's", 10), ("Django Unchained'", 10), ('Best Screenplay', 10), ('Most Words', 10), ('Pulp Fiction', 10), ('YESSSS', 10), ('Oscars  No 2012', 10)]
top results after merging:
[('Quentin Tarantino', 1247), ('Django Unchained', 230), ('WINNER', 40), ('Except', 40), ('QT', 21), ('tonight', 21), ('Will Lincoln', 21), ('Very', 10), ('2nd', 10), ('Surprised', 10)]

10
Predicting for: Best Motion Picture - Animated
true winner: Brave
award name reduced: ['animated']
num keywords to match: 1
num of matches: 147
top results:
[('Ralph', 72), ('Wreck

no results found or too few matches!!! add more keywords and reduce num_keywords_to_match!!!
num keywords to match: 3
award before ['actress', 'tv', 'series', 'drama']
expanded award with more keywards ['actress', 'tv', 'series', 'drama', 'television']
num of matches: 54
top results:
[('Claire Danes', 157), ('Homeland Best', 40), ('Homeland', 39), ('Damien Lewis', 11), ("Homeland' Best Actor", 10), ('Congrats Claire Danes', 10), ('Carrie', 10), ('Girls', 10), ('Lena Dunham', 4), ('Amy Poehler', 4), ('George Clooney', 4), ('Ten', 2), ('Well', 1), ('Dennis Quaid', 1), ('Kerry Washington', 1), ('Maggie Smith', 1), ('Julianne Moore', 1), ('Juliane Moore', 1), ('Jessica Lange', 1), ('Looking', 1), ('HOMELAND', 1), ('Congrats', 1)]
top results after merging:
[('Claire Danes', 1512), ('Homeland Best', 136), ('Damien Lewis', 11), ('Carrie', 10), ('Girls', 10), ('Lena Dunham', 4), ('Amy Poehler', 4), ('George Clooney', 4), ('Julianne Moore', 2), ('Ten', 2)]

20
Predicting for: Best Performance 